In [3]:
"""
=====================================================================
NOTEBOOK 5 : INTERFACE DE DÉMONSTRATION INTERACTIVE
Projet : Système de Recommandation MovieLens sur Amazon SageMaker
Auteur : Gninninmaguignon Silué
Date : Octobre 2025
=====================================================================
"""

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print(" INTERFACE DE DÉMONSTRATION INTERACTIVE")
print("=" * 70)

# ============================================
# PARTIE 1 : CHARGEMENT DU MODÈLE
# ============================================

print("\n Chargement du modèle et des données...")

device = torch.device('cpu')

# Architecture du modèle
class HybridRecommenderNet(nn.Module):
    def __init__(self, n_users, n_items, n_features, 
                 embedding_dim=128, hidden_dims=[256, 128, 64]):
        super(HybridRecommenderNet, self).__init__()
        
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        self.user_bn = nn.BatchNorm1d(embedding_dim)
        self.item_bn = nn.BatchNorm1d(embedding_dim)
        
        self.feature_fc = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(64)
        )
        
        total_input = embedding_dim * 2 + 64
        layers = []
        input_dim = total_input
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.BatchNorm1d(hidden_dim)
            ])
            input_dim = hidden_dim
        
        layers.append(nn.Linear(input_dim, 1))
        self.fc_layers = nn.Sequential(*layers)
    
    def forward(self, user, item, features):
        user_emb = self.user_embedding(user)
        item_emb = self.item_embedding(item)
        user_emb = self.user_bn(user_emb)
        item_emb = self.item_bn(item_emb)
        feat_emb = self.feature_fc(features)
        x = torch.cat([user_emb, item_emb, feat_emb], dim=1)
        output = self.fc_layers(x)
        return output.squeeze()

# Charger le checkpoint
checkpoint = torch.load('../models/saved_models/best_model.pth', map_location=device, weights_only=False)
n_users = checkpoint['n_users']
n_items = checkpoint['n_items']
n_features = checkpoint['n_features']

model = HybridRecommenderNet(n_users, n_items, n_features).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Charger les encoders et données
with open('../models/encoders/user_encoder.pkl', 'rb') as f:
    user_encoder = pickle.load(f)
with open('../models/encoders/item_encoder.pkl', 'rb') as f:
    item_encoder = pickle.load(f)

movies_meta = pd.read_csv("../data/processed/movies_metadata.csv")
users_meta = pd.read_csv("../data/processed/users_metadata.csv")
data_full = pd.read_csv("../data/processed/train_features.csv")

print("✅ Modèle et données chargés")
print(f"   RMSE: {checkpoint['rmse']:.4f}")
print(f"   MAE: {checkpoint['mae']:.4f}")

# ============================================
# PARTIE 2 : FONCTIONS UTILITAIRES
# ============================================

def get_user_profile(user_id_original):
    """Récupérer le profil d'un utilisateur"""
    user_data = users_meta[users_meta['user_id'] == user_id_original]
    if len(user_data) == 0:
        return None
    
    user_info = user_data.iloc[0]
    user_ratings = data_full[data_full['user_id'] == user_id_original]
    
    return {
        'user_id': int(user_id_original),
        'age': int(user_info['age']),
        'gender': user_info['gender'],
        'occupation': user_info['occupation'],
        'n_ratings': len(user_ratings),
        'avg_rating': float(user_ratings['rating'].mean()) if len(user_ratings) > 0 else 0
    }

def get_movie_info(item_id):
    """Récupérer les informations d'un film"""
    movie = movies_meta[movies_meta['item_id'] == item_id]
    if len(movie) == 0:
        return None
    
    movie_info = movie.iloc[0]
    genre_cols = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy',
                  'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                  'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                  'Thriller', 'War', 'Western']
    
    genres = [col for col in genre_cols if movie_info[col] == 1]
    item_ratings = data_full[data_full['item_id'] == item_id]
    
    return {
        'item_id': int(item_id),
        'title': movie_info['title'],
        'genres': genres,
        'n_ratings': len(item_ratings),
        'avg_rating': float(item_ratings['rating'].mean()) if len(item_ratings) > 0 else 0
    }

def recommend_top_k(user_id_original, top_k=10, exclude_rated=True):
    """Recommander les top-K films"""
    if user_id_original not in user_encoder.classes_:
        return None
    
    user_id_encoded = user_encoder.transform([user_id_original])[0]
    
    user_tensor = torch.tensor([user_id_encoded] * n_items, dtype=torch.long).to(device)
    item_tensor = torch.arange(n_items, dtype=torch.long).to(device)
    
    # Features
    user_data = data_full[data_full['user_id'] == user_id_original]
    if len(user_data) > 0:
        feature_cols = checkpoint['feature_cols']
        user_features_mean = user_data[feature_cols].mean().values
        features = np.tile(user_features_mean, (n_items, 1))
    else:
        features = np.zeros((n_items, n_features))
    
    features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
    
    with torch.no_grad():
        predictions = model(user_tensor, item_tensor, features_tensor)
        predictions = predictions.cpu().numpy()
    
    item_ids_original = item_encoder.inverse_transform(range(n_items))
    recommendations_df = pd.DataFrame({
        'item_id': item_ids_original,
        'predicted_rating': predictions
    })
    
    if exclude_rated:
        rated_items = data_full[data_full['user_id'] == user_id_original]['item_id'].values
        recommendations_df = recommendations_df[~recommendations_df['item_id'].isin(rated_items)]
    
    recommendations_df = recommendations_df.sort_values('predicted_rating', ascending=False)
    top_recommendations = recommendations_df.head(top_k)
    
    recommendations = []
    for _, row in top_recommendations.iterrows():
        movie_info = get_movie_info(row['item_id'])
        if movie_info:
            movie_info['predicted_rating'] = float(row['predicted_rating'])
            recommendations.append(movie_info)
    
    return recommendations

# ============================================
# PARTIE 3 : INTERFACE INTERACTIVE
# ============================================

print("\n" + "=" * 70)
print(" INTERFACE INTERACTIVE")
print("=" * 70)

# Style CSS
display(HTML("""
<style>
    .output-area {
        background: #f8f9fa;
        padding: 20px;
        border-radius: 10px;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 20px;
        border-radius: 10px;
        text-align: center;
        margin-bottom: 20px;
        font-size: 2rem;
        font-weight: bold;
    }
    .profile-card {
        background: white;
        padding: 15px;
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        margin-bottom: 15px;
        border-left: 4px solid #667eea;
    }
    .movie-card {
        background: white;
        padding: 12px;
        border-radius: 8px;
        margin-bottom: 10px;
        border-left: 4px solid #e50914;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .metric {
        font-size: 1.5rem;
        font-weight: bold;
        color: #667eea;
    }
</style>
"""))

# Widgets
all_users = sorted(data_full['user_id'].unique())

user_dropdown = widgets.Dropdown(
    options=all_users,
    value=all_users[0],
    description='Utilisateur:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

top_k_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    step=1,
    description='Top-K:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

exclude_checkbox = widgets.Checkbox(
    value=True,
    description='🚫 Exclure films déjà notés',
    style={'description_width': 'initial'}
)

button = widgets.Button(
    description='🚀 Générer Recommandations',
    button_style='success',
    layout=widgets.Layout(width='300px', height='40px')
)

output = widgets.Output()

# Fonction de callback
def on_button_click(b):
    with output:
        clear_output(wait=True)
        
        user_id = user_dropdown.value
        top_k = top_k_slider.value
        exclude_rated = exclude_checkbox.value
        
        # Header
        display(HTML(f"""
        <div class="header">
            🎬SYSTÈME DE RECOMMANDATION MOVIELENS
        </div>
        """))
        
        # Profil utilisateur
        profile = get_user_profile(user_id)
        
        if profile:
            gender_icon = "👨" if profile['gender'] == 'M' else "👩"
            display(HTML(f"""
            <div class="profile-card">
                <h2>👤 Profil Utilisateur #{profile['user_id']}</h2>
                <p>
                    {gender_icon} <strong>{profile['age']} ans</strong> | 
                    💼 <strong>{profile['occupation']}</strong>
                </p>
                <p>
                    📊 <span class="metric">{profile['n_ratings']}</span> films notés | 
                    ⭐ Note moyenne: <span class="metric">{profile['avg_rating']:.2f}</span>
                </p>
            </div>
            """))
            
            # Films préférés
            display(HTML("<h3>❤️ Films Préférés</h3>"))
            user_ratings = data_full[data_full['user_id'] == user_id].sort_values(
                'rating', ascending=False
            ).head(5)
            
            for idx, row in user_ratings.iterrows():
                movie = get_movie_info(row['item_id'])
                if movie:
                    genres_str = ', '.join(movie['genres'][:3])
                    display(HTML(f"""
                    <div class="movie-card">
                        <strong>{movie['title']}</strong><br>
                        ⭐ {row['rating']}/5 | 🎭 {genres_str}
                    </div>
                    """))
            
            # Recommandations
            display(HTML(f"<h3>🎯 Top {top_k} Recommandations</h3>"))
            
            recommendations = recommend_top_k(user_id, top_k=top_k, 
                                             exclude_rated=exclude_rated)
            
            if recommendations:
                for i, rec in enumerate(recommendations, 1):
                    genres_str = ', '.join(rec['genres'][:3])
                    stars = "⭐" * int(round(rec['predicted_rating']))
                    
                    display(HTML(f"""
                    <div class="movie-card">
                        <strong>#{i}. {rec['title']}</strong><br>
                        📊 Score prédit: <span class="metric">{rec['predicted_rating']:.2f}</span> {stars}<br>
                        🎭 Genres: {genres_str}<br>
                        📈 Note moyenne: {rec['avg_rating']:.2f}/5 ({rec['n_ratings']} votes)
                    </div>
                    """))
                
                # Graphique de distribution
                scores = [r['predicted_rating'] for r in recommendations]
                
                fig, ax = plt.subplots(figsize=(10, 4))
                ax.hist(scores, bins=10, color='#667eea', edgecolor='black', alpha=0.7)
                ax.set_title('Distribution des Scores Prédits', fontsize=14, fontweight='bold')
                ax.set_xlabel('Score Prédit')
                ax.set_ylabel('Fréquence')
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            else:
                display(HTML("<p style='color: red;'>❌ Impossible de générer des recommandations</p>"))
        else:
            display(HTML("<p style='color: red;'>❌ Utilisateur introuvable</p>"))

button.on_click(on_button_click)

# Affichage de l'interface
print("\n Interface prête !")
print("\n Utilisez les widgets ci-dessous pour tester le système:")

display(widgets.VBox([
    widgets.HTML("<h2 style='color: #667eea;'>⚙️ Configuration</h2>"),
    user_dropdown,
    top_k_slider,
    exclude_checkbox,
    button,
    output
]))

# ============================================
# PARTIE 4 : EXEMPLES DE TESTS
# ============================================

print("\n" + "=" * 70)
print(" EXEMPLES DE TESTS RAPIDES")
print("=" * 70)

# Test avec quelques utilisateurs aléatoires
test_users = np.random.choice(all_users, 3, replace=False)

for user_id in test_users:
    profile = get_user_profile(user_id)
    recs = recommend_top_k(user_id, top_k=5)
    
    print(f"\n👤 Utilisateur {user_id}: {profile['age']}ans, {profile['occupation']}")
    print(f"   Top 5 recommandations:")
    for i, rec in enumerate(recs[:5], 1):
        print(f"      {i}. {rec['title'][:50]:50s} ({rec['predicted_rating']:.2f}⭐)")

print("\n" + "=" * 70)
print(" INTERFACE ET TESTS COMPLÉTÉS")
print("=" * 70)

print("\n Conseils:")
print("   • Utilisez les widgets ci-dessus pour tester différents utilisateurs")
print("   • Prenez des captures d'écran pour votre rapport")
print("   • Analysez la cohérence des recommandations avec le profil")

print("\n🚀 PROCHAINE ÉTAPE: Génération du Rapport Final")
print("=" * 70)

 INTERFACE DE DÉMONSTRATION INTERACTIVE

 Chargement du modèle et des données...
✅ Modèle et données chargés
   RMSE: 0.6247
   MAE: 0.4492

 INTERFACE INTERACTIVE



 Interface prête !

 Utilisez les widgets ci-dessous pour tester le système:



 EXEMPLES DE TESTS RAPIDES

👤 Utilisateur 174: 30ans, administrator
   Top 5 recommandations:
      1. Pather Panchali (1955)                             (4.75⭐)
      2. Golden Earrings (1947)                             (4.66⭐)
      3. Shawshank Redemption, The (1994)                   (4.56⭐)
      4. Aiqing wansui (1994)                               (4.55⭐)
      5. Santa with Muscles (1996)                          (4.52⭐)

👤 Utilisateur 472: 24ans, student
   Top 5 recommandations:
      1. Pather Panchali (1955)                             (5.01⭐)
      2. Golden Earrings (1947)                             (4.83⭐)
      3. Schindler's List (1993)                            (4.78⭐)
      4. Santa with Muscles (1996)                          (4.78⭐)
      5. Usual Suspects, The (1995)                         (4.74⭐)

👤 Utilisateur 580: 16ans, student
   Top 5 recommandations:
      1. Pather Panchali (1955)                             (4.58⭐)
      2. Golden Earrings (1947)    